In [1]:
from glob import glob

from hex import *

import re
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import layers, utils
from keras import regularizers

/home/none/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def getBoard(board):    
   return np.take(board.state[1:14], range(1,14), axis=1)

In [3]:
def makeInputOutput(moveList, winner, scraped):
    '''
    moveList - string representing all move in a game
    scraped - True - this movelist was scraped False - a movelist from a hex game
    winner - string - white or black
    '''
    outputWinner = [1,0]
    if (winner == 'white'):
        outputWinner = [0,1]
    inputBoard = []
    outputWin = []
    if scraped:
        splitOn = r"([a-z]+)([0-9]+)"
    else:
        splitOn = r"([0-9]+)([a-z]+)"
    black = True
    tempBoard = Board()
    moves = re.findall(splitOn, moveList)
    for index, move in enumerate(moves):
        inputBoard.append(getBoard(tempBoard))
        #D sun stores number then letter
        if scraped:
          move = move[::-1]
        # move is in form (number, letter)
        # Change letter to integer
        letterToNumber = tempBoard.col_to_number[move[1]]
        # Cast string number to integer
        move = (int(move[0]), letterToNumber)
        # If * mirror the move set the first player to white
        if '*' in moveList and index==0:
            #mirror the first move
            black = not(black)
            move = (move[1], move[0])
        #Out of 13*13= 169 possible board squares
        moveIntegerized = 13*(int(move[1])-1) + move[0]
        # Store which square the piece went
        outputWin.append(outputWinner)
        # Append piece to the board 1 is black 2 is white
        tempBoard.addPiece(move, 1 if black else 2)
        black = not(black)
            
    return inputBoard[1:], outputWin[1:]

In [4]:
data = pd.read_csv('finaldataframe.csv', index_col=0)

In [5]:
def makeInputOutputTotal(moveLists, winners, scraped=True):
    total = map(lambda args: makeInputOutput(args[0], args[1], args[2]),
                filter(lambda args: len(args[1])<12,zip(moveLists, winners, [scraped]*len(winners))))
    board_states_cumulative, winners_cumulative = zip(*total)
    board_states_cumulative = [val for sublist in board_states_cumulative for val in sublist]
    winners_cumulative = [val for sublist in winners_cumulative for val in sublist]
    return (board_states_cumulative,
            winners_cumulative)
    

In [6]:
data.head(10)

,gid,black,white,black_rating,white_rating,move_list,winner
0,1939599,fastplayer,Maciej Celuch,1806,2688,g3*e9i10f7d12b12c11d11c12f11g12e12g5j4h7c9h3f5...,white
1,1938419,MoHex3H,Maciej Celuch,1500,2688,h12e4f4i5h6e9i7i6h7g5k5k4j4j5h5i3i4k2j2j3f7e6h...,white
2,1938296,fastplayer,Maciej Celuch,1806,2688,g7*f7d10i8j11f8g6i5h4j3j4i4g8j6j5e10f11i6h9g10...,white
3,1938295,Jose Maria Grau Ribas,Maciej Celuch,2083,2688,a4*c3i10f8j2k10k11i3e6g6f10e10f9e9d12e11e12f11...,white
4,1938291,MoHex3H,Maciej Celuch,1500,2688,g7*e8e4c5f6j5i10h9g11g6f9h11h10d10d9c8b12f10g9...,white
5,1909790,Maciej Celuch,Yorge,2688,1520,h3*d5d3j9h6j4j11e9f8k6g11d8,black
6,1877598,zastrzyk,Maciej Celuch,1751,2688,a8*i5i10k10k11e9c11g10f8e6f9h8h9g9h7,white
7,1877596,Maciej Celuch,Peter Koning,2688,1694,a3,black
8,1877594,Daniel Sepczuk,Maciej Celuch,2386,2688,c12e4i5j4d5e9e5d3b3d4b6b4i4h8j9h7f7e10,white
9,1877591,Maciej Celuch,Szymon D,2688,1531,m4,black


In [7]:
moveLists = list(data['move_list'])
winners = list(data['winner'])


In [8]:
board_states_cumulative, winners_cumulative = makeInputOutputTotal(moveLists, winners)

In [9]:
board_states_cumulative[0]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int8)

In [10]:
winners_cumulative[0]

[0, 1]

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(board_states_cumulative, winners_cumulative, test_size = 0.2)

In [12]:
X_train = [X.reshape(13,13,1) for X in X_train]
X_test = [X.reshape(13,13,1) for X in X_test]

In [13]:
regularizationParam = 0.01
model = tf.keras.Sequential([
    layers.Conv2D(256, kernel_size=(3, 3), strides=(1, 1),
                  kernel_regularizer=regularizers.l2(regularizationParam),
                  padding='same',
                  activation='linear',
                  input_shape=(13, 13, 1)),
    layers.BatchNormalization(axis=1, momentum=0.99, epsilon=0.001),
    layers.LeakyReLU(alpha=0.3),
    layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
    layers.Conv2D(64, (3, 3),
                  padding='same',
                  activation='relu',
                  kernel_regularizer=regularizers.l2(regularizationParam)),
    layers.Conv2D(64, (3, 3),
                  padding='same',
                  activation='relu',
                  kernel_regularizer=regularizers.l2(regularizationParam)),
    layers.MaxPool2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu',kernel_regularizer=regularizers.l2(regularizationParam)),
    layers.Dense(50, activation='tanh',kernel_regularizer=regularizers.l2(regularizationParam)),
    layers.Dense(2, activation='softmax',kernel_regularizer=regularizers.l2(regularizationParam))
])

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [14]:
len(X_train)

1145668

In [ ]:
for _ in range(10):
    inds = list(range(1145668))
    np.random.shuffle(inds)
    x_train_tmp = [X_train[i] for i in inds[:1000]]
    y_train_tmp = [y_train[i] for i in inds[:1000]]
    model.fit(np.array(x_train_tmp), np.array(y_train_tmp), verbose=1, epochs=3)#, steps_per_epoch=3) more samples less epochs

Epoch 1/3
1000/1000 [==============================] - 4s 4ms/step - loss: 4.5716 - acc: 0.4940
Epoch 2/3
1000/1000 [==============================] - 3s 3ms/step - loss: 3.0811 - acc: 0.4840
Epoch 3/3
1000/1000 [==============================] - 3s 3ms/step - loss: 2.2703 - acc: 0.5070
Epoch 1/3
1000/1000 [==============================] - 3s 3ms/step - loss: 1.8088 - acc: 0.4960
Epoch 2/3
1000/1000 [==============================] - 3s 3ms/step - loss: 1.5236 - acc: 0.4640
Epoch 3/3
1000/1000 [==============================] - 3s 3ms/step - loss: 1.3292 - acc: 0.4840
Epoch 1/3
1000/1000 [==============================] - 3s 3ms/step - loss: 1.1899 - acc: 0.4890
Epoch 2/3
1000/1000 [==============================] - 3s 3ms/step - loss: 1.0871 - acc: 0.5080
Epoch 3/3
1000/1000 [==============================] - 3s 3ms/step - loss: 1.0243 - acc: 0.4800
Epoch 1/3
1000/1000 [==============================] - 3s 3ms/step - loss: 0.9710 - acc: 0.4860
Epoch 2/3
1000/1000 [===================

In [ ]:
#model.fit(np.array(X_train), np.array(y_train), verbose=1, epochs=1, batch_size=1)

Epoch 1/1
  10873/1585914 [..............................] - ETA: 4:04:59 - loss: 3.2781e-04 - acc: 0.9999

In [18]:
scores = model.evaluate(np.array(X_test[:1000]), np.array(y_test[:1000]))

print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

1000/1000 [==============================] - 2s 2ms/step

acc: 54.10%


In [177]:
len(y_test)

286417

In [178]:
len(X_test)

1145668